## **# Step 1: Load the Dataset**

In [ ]:
import pandas as pd
import re
# Load dataset
file_path = "/content/sample_data/IMDB Dataset.csv"
df = pd.read_csv(file_path)

# Display first few rows
df.head()

## **# Step 2: Text Preprocessing**

In [ ]:
# Function to clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)  # Remove special characters
    return text

# Apply text cleaning
df['review'] = df['review'].astype(str).apply(clean_text)

# Convert sentiment labels to numeric values
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Display cleaned data
df.head()


##**# Step 3: Import Libraries for Data Splitting**

In [ ]:
from sklearn.model_selection import train_test_split
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)


## **# Step 4: Import Libraries for Feature Extraction**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Convert text to TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


## **# Step 5: Import Libraries for Model Training**

In [ ]:
from sklearn.linear_model import LogisticRegression
# Train Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)


##**# Step 6: Import Libraries for Model Evaluation**

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
# Make predictions
y_pred = model.predict(X_test_tfidf)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))


##**# Step 7: Import Library for Model Saving**

In [ ]:
import pickle
# Save model and vectorizer
pickle.dump(model, open("sentiment_model.pkl", "wb"))
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))


## **# Step 8: Import Libraries for Flask Deployment**

In [ ]:
from flask import Flask, request, jsonify
# Initialize Flask app
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json['review']
    data_clean = clean_text(data)
    data_tfidf = vectorizer.transform([data_clean])
    prediction = model.predict(data_tfidf)[0]
    sentiment = "positive" if prediction == 1 else "negative"
    return jsonify({'sentiment': sentiment})

if __name__ == '__main__':
    app.run(debug=True)
